### Pruebas con la librería pandas

In [24]:
import pandas as pd
from os.path import isfile
import sqlite3 as db

In [2]:
dt = pd.read_excel("pedidos2.xlsx")
dt.head()

,idpedido,cliente,idempleado,idempresa,importe,pais
0,10248,WILMK,5,3,32.38,Finlandia
1,10249,TOMSP,6,1,11.61,Alemania
2,10250,HANAR,4,2,65.83,Brasil
3,10251,VICTE,3,1,41.34,Francia
4,10252,SUPRD,4,2,51.30,Belgica


In [3]:
dt.tail()

,idpedido,cliente,idempleado,idempresa,importe,pais
825,11073,PERIC,2,2,24.95,Mexico
826,11074,SIMOB,7,2,18.44,Dinamarca
827,11075,RICSU,8,2,6.19,Suiza
828,11076,BONAP,4,2,38.28,Francia
829,11077,RATTC,1,2,8.53,Estados Unidos


In [18]:
dt2 = dt[(dt.pais == 'Suiza') & (dt.importe > 130)]
dt2.to_json("paises/suiza.json", orient="records")

In [4]:
dt.importe

0      32.38
1      11.61
2      65.83
3      41.34
4      51.30
       ...  
825    24.95
826    18.44
827     6.19
828    38.28
829     8.53
Name: importe, Length: 830, dtype: float64

In [5]:
dt.shape

(830, 6)

In [6]:
dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   idpedido    830 non-null    int64  
 1   cliente     830 non-null    object 
 2   idempleado  830 non-null    int64  
 3   idempresa   830 non-null    int64  
 4   importe     830 non-null    float64
 5   pais        830 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 39.0+ KB


In [7]:
dt.describe()

,idpedido,idempleado,idempresa,importe
count,830.000000,830.000000,830.000000,830.000000
mean,10662.500000,4.403614,2.008434,78.244205
std,239.744656,2.499648,0.778899,116.779294
min,10248.000000,1.000000,1.000000,0.020000
25%,10455.250000,2.000000,1.000000,13.380000
50%,10662.500000,4.000000,2.000000,41.360000
75%,10869.750000,7.000000,3.000000,91.430000
max,11077.000000,9.000000,3.000000,1007.640000


In [8]:
dt.pais.unique()

array(['Finlandia', 'Alemania', 'Brasil', 'Francia', 'Belgica', 'Suiza',
       'Venezuela', 'Austria', 'Mexico', 'Estados Unidos', 'Suecia',
       'Italia', 'Espanya', 'Reino Unido', 'Irlanda', 'Portugal',
       'Canada', 'Dinamarca', 'Polonia', 'Noruega', 'Argentina'],
      dtype=object)

In [11]:
L = sorted(dt.pais.unique())
L[:5]

['Alemania', 'Argentina', 'Austria', 'Belgica', 'Brasil']

In [12]:
len(L)

21

In [ ]:
# Número de pedidos por país
dt.pais.value_counts()

Exportar un fichero Excel o CSV por cada pais que tenemos en el fichero

In [21]:
# Quitar repetidos:
L = sorted(dt.pais.unique())

# Generar nuevas columnas en el dataframe
dt["iva"] = round(dt.importe * 0.21, 2)
dt["total"] = round(dt.importe + dt.iva, 2)

# Recorrer la colección de paises
for pais in L:
    
    # Montar el path del fichero destino
    path = f"paises/{pais}.csv"
    
    # Filtrar por pais
    dt2 = dt[dt.pais == pais]
    print(dt2.shape, " en el fichero: ", path)
    
    # Exportar al fichero:
    dt2.to_csv(path, sep=";", decimal=",", index=False)

(122, 8)  en el fichero:  paises/Alemania.csv
(16, 8)  en el fichero:  paises/Argentina.csv
(40, 8)  en el fichero:  paises/Austria.csv
(19, 8)  en el fichero:  paises/Belgica.csv
(83, 8)  en el fichero:  paises/Brasil.csv
(30, 8)  en el fichero:  paises/Canada.csv
(18, 8)  en el fichero:  paises/Dinamarca.csv
(23, 8)  en el fichero:  paises/Espanya.csv
(123, 8)  en el fichero:  paises/Estados Unidos.csv
(23, 8)  en el fichero:  paises/Finlandia.csv
(76, 8)  en el fichero:  paises/Francia.csv
(19, 8)  en el fichero:  paises/Irlanda.csv
(28, 8)  en el fichero:  paises/Italia.csv
(28, 8)  en el fichero:  paises/Mexico.csv
(6, 8)  en el fichero:  paises/Noruega.csv
(6, 8)  en el fichero:  paises/Polonia.csv
(13, 8)  en el fichero:  paises/Portugal.csv
(55, 8)  en el fichero:  paises/Reino Unido.csv
(38, 8)  en el fichero:  paises/Suecia.csv
(18, 8)  en el fichero:  paises/Suiza.csv
(46, 8)  en el fichero:  paises/Venezuela.csv


In [29]:
sql = """select c.nombre as nombre_cat, p.nombre as nombre_prod, p.precio
from categorias c 
inner join productos p on c.id = p.idcategoria
order by 1,2"""

In [31]:
path = "empresa3.db"
if isfile(path):
    con = db.connect(path)
    
    dt2 = pd.read_sql(sql, con)
    print(dt2.head())
    
    dt2.to_html("paises/productos.html", index=False)
    
else:
    raise FileNotFoundError(f"No existe el fichero: {path}" )
    

  nombre_cat            nombre_prod     precio
0     Carnes        Buey Mishi Kobe  117.90400
1     Carnes  Cordero Alice Springs   47.40470
2     Carnes      Empanada de carne   39.86860
3     Carnes      Empanada de cerdo    9.05552
4     Carnes             Pate chino   24.00000


In [33]:
# Extraer datos en Python  de una BD pero sin pandas

con = db.connect(path)
sql = """select c.nombre as nombre_cat, p.nombre as nombre_prod, p.precio
from categorias c 
inner join productos p on c.id = p.idcategoria
order by 1,2"""
cur = con.cursor()
cur.execute(sql)

for t in cur.fetchall():
    if t[2] > 100:
        print(t)
    
cur.close()
con.close()

('Carnes', 'Buey Mishi Kobe', 117.904)
('Carnes', 'Salchicha Thuringer', 150.468)
('bebidas', 'Azafrán Rojo', 250.0)
('comidas', 'Vino Cote de Blaye', 320.286)
